# [Matrix multiplication from foundations](https://github.com/fastai/course-v3/blob/master/nbs/swift/01_matmul.ipynb)

## Initalize Swift

In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

Installing packages:
	.package(path: "/notebooks/fastai_p2_v3/FastaiNotebook_00_load_data")
		FastaiNotebook_00_load_data
With SwiftPM flags: []
Working in: /tmp/tmpk0eydjbo/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
//export
import Path
import TensorFlow

In [3]:
import FastaiNotebook_00_load_data

In [4]:
let xTrain = Tensor<Float>(randomNormal: [5, 784])
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784)
print(weights[0])

[ 0.05203135, 0.033128623, 0.034648303, 0.012412762, 0.014186986,  0.01595626, -0.03589106,
 0.023133205, -0.02464643, 0.031687967]


## Build Matmul

In [5]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmul(a: [Float], b: [Float], aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    for i in 0 ..< aDims.0 {
        for j in 0 ..< bDims.1 {
            for k in 0 ..< aDims.1 {
                res[i*bDims.1+j] += a[i*aDims.1+k] * b[k*bDims.1+j]
            }
        }
    }
    return res
}

In [6]:
let flatA = xTrain[0..<5].scalars
let flatB = weights.scalars
let (aDims,bDims) = ((5, 784), (784, 10))

In [7]:
var resultArray = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)

In [8]:
time(repeating: 100) {
    _ = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.09481812999999999 ms,   min: 0.082456 ms,   max: 0.140812 ms


## [UnsafePointer](https://www.fast.ai/2019/01/10/swift-numerics/)

### Using raw C Pointer

In [9]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmulUnsafe(a: UnsafePointer<Float>, b: UnsafePointer<Float>, aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    res.withUnsafeMutableBufferPointer { res in 
        for i in 0 ..< aDims.0 {
            for j in 0 ..< bDims.1 {
                for k in 0 ..< aDims.1 {
                    res[i*bDims.1+j] += a[i*aDims.1+k] * b[k*bDims.1+j]
                }
            }
        }
    }
    return res
}

### Declaration
```swift
mutating func withUnsafeMutableBufferPointer<R>(_ body: (inout UnsafeMutableBufferPointer<Element>) throws -> R) rethrows -> R
```

If you need an array that is preinitialized with a fixed number of default values, use the [Array](https://developer.apple.com/documentation/swift/array)(repeating:count:) initializer.

```swift
var digitCounts = Array(repeating: 0, count: 10)
print(digitCounts)
// Prints "[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
```

[Array.withUnsafeMutableBufferPointer](https://developer.apple.com/documentation/swift/array/2994773-withunsafemutablebufferpointer)

```swift
var numbers = [1, 2, 3, 4, 5]
numbers.withUnsafeMutableBufferPointer { buffer in
    for i in stride(from: buffer.startIndex, to: buffer.endIndex - 1, by: 2) {
        buffer.swapAt(i, i + 1)
    }
}
print(numbers)
// Prints "[2, 1, 4, 3, 5]"
```

In [10]:
time(repeating: 100) {
    _ = swiftMatmulUnsafe(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.04092096 ms,   min: 0.037303 ms,   max: 0.067506 ms


## Using C APIs

In [11]:
import Glibc

let ptr : UnsafeMutableRawPointer = malloc(42)

print("☠️☠️ Uninitialized garbage =", ptr.load(as: UInt8.self))

free(ptr)

☠️☠️ Uninitialized garbage = 176


## Working with Tensor

In [12]:
var bias = Tensor<Float>(zeros: [10])

let m1 = Tensor<Float>(randomNormal: [5, 784])
let m2 = Tensor<Float>(randomNormal: [784, 10])

In [13]:
print("m1: ", m1.shape)
print("m2: ", m2.shape)

m1:  [5, 784]
m2:  [784, 10]


In [14]:
let small = Tensor<Float>([[1, 2],
                           [3, 4]])

print("🔢2x2:\n", small)

🔢2x2:
 [[1.0, 2.0],
 [3.0, 4.0]]


In [15]:
print("⊞ matmul:\n",  matmul(small, small))
print("\n⊞ again:\n", small • small)

⊞ matmul:
 [[ 7.0, 10.0],
 [15.0, 22.0]]

⊞ again:
 [[ 7.0, 10.0],
 [15.0, 22.0]]


In [16]:
var m = Tensor([1.0, 2, 3, 4, 5, 6, 7, 8, 9]).reshaped(to: [3, 3])
print(m)

[[1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0],
 [7.0, 8.0, 9.0]]


In [17]:
sqrt((m * m).sum())

16.881943016134134


## Matmul using Tensor

In [18]:
func tensorMatmul(_ a: Tensor<Float>, _ b: Tensor<Float>) -> Tensor<Float> {
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])

    for i in 0 ..< a.shape[0] {
        for j in 0 ..< b.shape[1] {
            for k in 0 ..< a.shape[1] {
                res[i, j] += a[i, k] * b[k, j]
            }
        }
    }
    return res
}

_ = tensorMatmul(m1, m2)

In [19]:
time { 
    let tmp = tensorMatmul(m1, m2)
    
    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 4124.909725 ms,   min: 4124.909725 ms,   max: 4124.909725 ms


## Vectorize the inner loop into a multiply + sum

In [20]:
func elementWiseMatmul(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    let (ar, ac) = (a.shape[0], a.shape[1])
    let (br, bc) = (b.shape[0], b.shape[1])
    var res = Tensor<Float>(zeros: [ac, br])
    
    for i in 0 ..< ar {
        let row = a[i]
        for j in 0 ..< bc {
            res[i, j] = (row * b.slice(lowerBounds: [0,j], upperBounds: [ac,j+1]).squeezingShape(at: 1)).sum()
        }
    }
    return res
}

_ = elementWiseMatmul(m1, m2)

In [21]:
time { 
    let tmp = elementWiseMatmul(m1, m2)

    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 369.139894 ms,   min: 369.139894 ms,   max: 369.139894 ms


## Vectorize the inner two loops with broadcasting

In [22]:
func broadcastMatmult(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0] {
        res[i] = (a[i].expandingShape(at: 1) * b).sum(squeezingAxes: 0)
    }
    return res
}

_ = broadcastMatmult(m1, m2)

In [23]:
time(repeating: 100) {
    let tmp = broadcastMatmult(m1, m2)

    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 0.5405445799999998 ms,   min: 0.447108 ms,   max: 0.909627 ms


## Vectorize the whole thing with one Tensorflow op

In [24]:
time(repeating: 100) { _ = m1 • m2 }

average: 0.01687047 ms,   min: 0.016628 ms,   max: 0.022531 ms


## Tensorflow vectorizes, parallelizes, and scales

In [25]:
func timeMatmulTensor(size: Int) {
    var matrix = Tensor<Float>(randomNormal: [size, size])
    print("\n\(size)x\(size):\n  ⏰", terminator: "")
    time(repeating: 10) { 
        let matrix = matrix • matrix 
        _ = matrix[0, 0].scalar
    }
}

timeMatmulTensor(size: 1)     // Tiny
timeMatmulTensor(size: 10)    // Bigger
timeMatmulTensor(size: 100)   // Even Bigger
timeMatmulTensor(size: 1000)  // Biggerest
timeMatmulTensor(size: 5000)  // Even Biggerest


1x1:
  ⏰average: 0.23084360000000004 ms,   min: 0.188788 ms,   max: 0.249671 ms

10x10:
  ⏰average: 0.2308456 ms,   min: 0.144309 ms,   max: 0.263194 ms

100x100:
  ⏰average: 0.15936880000000003 ms,   min: 0.152974 ms,   max: 0.174673 ms

1000x1000:
  ⏰average: 0.4983167999999999 ms,   min: 0.474408 ms,   max: 0.525607 ms

5000x5000:
  ⏰average: 36.1017642 ms,   min: 32.385793 ms,   max: 43.31702 ms


In [26]:
func timeMatmulSwift(size: Int, repetitions: Int = 10) {
    var matrix = Tensor<Float>(randomNormal: [size, size])
    let matrixFlatArray = matrix.scalars

    print("\n\(size)x\(size):\n  ⏰", terminator: "")
    time(repeating: repetitions) { 
       _ = swiftMatmulUnsafe(a: matrixFlatArray, b: matrixFlatArray, aDims: (size,size), bDims: (size,size))
    }
}

timeMatmulSwift(size: 1)     // Tiny
timeMatmulSwift(size: 10)    // Bigger
timeMatmulSwift(size: 100)   // Even Bigger
timeMatmulSwift(size: 1000, repetitions: 1)  // Biggerest

print("\n5000x5000: skipped, it takes tooo long!")


1x1:
  ⏰average: 0.00035729999999999996 ms,   min: 0.000314 ms,   max: 0.000615 ms

10x10:
  ⏰average: 0.004068500000000001 ms,   min: 0.003981 ms,   max: 0.00432 ms

100x100:
  ⏰average: 1.6743956999999998 ms,   min: 0.986649 ms,   max: 3.275343 ms

1000x1000:
  ⏰average: 1265.503771 ms,   min: 1265.503771 ms,   max: 1265.503771 ms

5000x5000: skipped, it takes tooo long!


In [27]:
withDevice(.cpu) {
    timeMatmulTensor(size: 1)     // Tiny
    timeMatmulTensor(size: 10)    // Bigger
    timeMatmulTensor(size: 100)   // Even Bigger
    timeMatmulTensor(size: 1000)  // Biggerest
    timeMatmulTensor(size: 5000)  // Even Biggerest
}


1x1:
  ⏰average: 0.0252171 ms,   min: 0.023791 ms,   max: 0.031558 ms

10x10:
  ⏰average: 0.0248304 ms,   min: 0.023232 ms,   max: 0.032301 ms

100x100:
  ⏰average: 0.20601060000000002 ms,   min: 0.163757 ms,   max: 0.25692 ms

1000x1000:
  ⏰average: 4.9400257000000005 ms,   min: 4.447461 ms,   max: 5.746169 ms

5000x5000:
  ⏰average: 683.6468265000001 ms,   min: 643.453733 ms,   max: 735.877237 ms


### Raw Tensor Operations

### [Protocol Buffer](https://developers.google.com/protocol-buffers/)

### [Tutorial on Raw operators](https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/raw_tensorflow_operators.ipynb)

In [28]:
//export
public extension StringTensor {
    // Read a file into a Tensor.
    init(readFile filename: String) {
        self.init(readFile: StringTensor(filename))
    }
    init(readFile filename: StringTensor) {
        self = Raw.readFile(filename: filename)
    }

    // Decode a StringTensor holding a JPEG file into a Tensor<UInt8>.
    func decodeJpeg(channels: Int = 0) -> Tensor<UInt8> {
        return Raw.decodeJpeg(contents: self, channels: Int64(channels), dctMethod: "") 
    }
}

## Export

In [29]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"01_matmul.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
